# Capitulo 7: Optimizing and Tuning Spark Applications

Iremos leyendo y realizando los ejemplos del capitulo 7 del libro, complementandolo con unos actividades sobre los mismos ejemplos.

##### Siempre debemos iniciar una instancia SparkSession al principio. 

Por lo que antes de comenzar crearemos la SparkSession correspondiente:

In [46]:
import pyspark
from pyspark.sql import SparkSession

# Creamos SparkSession
spark = SparkSession\
    .builder\
    .appName("LibroSpark_cap7")\
    .getOrCreate()

## Optimizing and Tuning Spark for Efficiency

En este capitulo trataremos de optimizar y afinar la configuración de Spark, mirando diferentes configuraciones y estrategias para su optimización e inspecionando la UI de Spark para ver el proceso.

De las muchas congiguraciones para afinar Saprk veremos las más comunes, para ver una lista agrupada y completa se recomienda ver la documentación. https://spark.apache.org/docs/latest/configuration.html

### Viewing and Setting Apache Spark Configurations

Existen 3 maneras para obtener y establecer (setear) las Spark properties.

1. A través de un conjunto de archivos de configuración que se encuentran en la SPARK_HOME en la carpeta de instalación de Spark (**C:/spark/conf**). Sobe estos archivos de configuracion (conf/spark-defaults.conf.template, conf/ log4j.properties.template, y conf/spark-env.sh.template) se pueden cambiar los valores por defecto y guardarlos sin el sufijo ".template" para indicarle a Spark que utilice estos nuevos valores.

2. Tambien podemos especificar en las diferentes lineas/opciones de la Spark Aplication las configuraciones DIRECTAMENTE al utilizar spark-submit (consola), usando --conf, como por ejemplo:

spark-submit --conf spark.sql.shuffle.partitions=5 --conf "spark.executor.memory=2g" --class main.scala.chapter7.SparkConfig_7_1 jars/mainscala-chapter7_2.12-1.0.jar

3. Por ultimo para la tercera opción, se puede hacer de manera porgramática con la spark-shell/notebook. La API es el metodo primario de configuración, por medio de la SparkSession puedes acceder a la mayoría de configuraciones de Spark. Podemos ver en el siguiente ejemplo con SparkSQL la configuración de Spark lanzado en modo local en nuestro localhost

De manera alternativa, puedes acceder a la configuración actual de Spark a través de la Spark UI's, viendose esto más adelante.

In [7]:
spark.sql("SET -v").select("key", "value").show(n=15, truncate=False)

+-----------------------------------------------------------+----------------------------------------------------------------+
|key                                                        |value                                                           |
+-----------------------------------------------------------+----------------------------------------------------------------+
|spark.sql.adaptive.advisoryPartitionSizeInBytes            |<value of spark.sql.adaptive.shuffle.targetPostShuffleInputSize>|
|spark.sql.adaptive.coalescePartitions.enabled              |true                                                            |
|spark.sql.adaptive.coalescePartitions.initialPartitionNum  |<undefined>                                                     |
|spark.sql.adaptive.coalescePartitions.minPartitionNum      |<undefined>                                                     |
|spark.sql.adaptive.enabled                                 |false                                             

Para modificar una configuración de manera programática, primero checkeamos las si la propiedad es modificable con:

**spark.conf.isModifiable("<config_name>")**

que nos devolverá True si se puede modificar o False en caso conrrario. Utilizamos **set** cuando queramos setear o modificar valores de configuración

In [14]:
# miramos si la configuración "spark.sql.shuffle.partitions" se puede modificar
spark.conf.isModifiable("spark.sql.shuffle.partitions")


True

In [12]:
# Obtenemos con get el numero de shuffle partitions por defecto
spark.conf.get("spark.sql.shuffle.partitions")
# Seteamos un nuevo numero de shuffle partitions y verificamos el cambio
spark.conf.set("spark.sql.shuffle.partitions", 5)
spark.conf.get("spark.sql.shuffle.partitions")


'5'

De entre todas las formas de modificar las Spark porperties, un **orden de precedencia** determina qué valores se respetan. Todo lo definido en spark-defaults.conf será tenido en cuenta primero, seguido de la parte de --conf (uso de spark-submit) y finalmente el uso de set vía Spark Session en la Spark aplication.

Todas estas propiedades se fusionarán, teniendo preferencia cualquier propiedad duplicada que se restablezca en la Spark Aplication que tendrá prioridad. Del mismo modo, los valores suministrados en la línea de comandos sustituirán a los ajustes
en el archivo de configuración spark-defaults.conf, siempre que no se sobrescriban en la aplicación misma.

Recordamos que este tipo de prácticas tienen como finalidad mejorar el rendimiento y maximizar los recursos del cluster para que Spark se adapte a cargas de trabajo a gran escala.

## Scaling Spark for Large Workloads

Las grandes cargas de trabajo (large workloads) en spark se suelen llevar a cabo por lotes o batch jobs (Algunos se ejecutan periodicamente y otros son programados a intervalos regulares). En estos jobs son procesados enormes cantidades de datos como terabytes. 

Para evitar fallos en los jobs debido a la falta de recursos o degradación del rendimiento existen configuraciones de Spark que puedes habilitar o modificar. Estas afectan a los 3 componentes principales de Spark: Spark Driver, Executor/s y el el Shuffle service que se lleva a cabo en el executor.

Recordamos que el Spark driver es el encargado de coordinarse con el cluster manager para lanzar los executords en un cluster y fijar las tareas en el. Con las large workloads tienes cientos de tareas. En esta sección, explicaremos algunas configuraciones que puedes ajustar o activar para optimizar los recursos, paralelizar tareas y evitar cuellos de botella en la ejecución de numerosas tareas. Algunas de estas recomendaciones han sido descubiertas por companias como Facebook, que utiliza Spark con terabytes.

### Static versus dynamic resource allocation
https://spark.apache.org/docs/latest/configuration.html#dynamic-allocation

Cuando se especifica los recursos en la linea de comandos --conf usando el spark-submit, tu acotas el límite de estos, ya que hay más recursos que serán necesarios despues para llevar a cabo las tareas en el driver acorde con large workloads. De esta manera se **establecen los recursos de manera Estática** y Spark no puede asignar recursos extra.

Por el contrario, si utilizas la **configuración de asignación dinámica de recursos**, el Spark Driver puede solicitar más o menos recursos según si el flujo de demanda de large workloads aumenta o disminuye. Gracias a esto, se pueden acomodar los picos repentinos de trabajos.

Uno de los casos de uso puede ser cuando utilizamos Spark Streaming, donde el flujo de datos puede ser desigual, o cuando tienes un alto volumen de querys SQL que pueden durar horas.  Activar la asignación dinámica permite a Spark una mejor utilización de los recursos, liberando a los Executors cuando no están en uso y adquiriendo otros nuevos cuando se necesitan.

Para activar la configuración dinámica, puedes utilizar settings como el siguiente ejemplo:

#. Por defecto esta la asignación dinámica suele estar desactivada/false

spark.dynamicAllocation.enabled true

#. Si la cola de tareas aumenta, se aumentan los executors cuando el tiempo de espera sea de más de 1 minuto

spark.dynamicAllocation.schedulerBacklogTimeout 1m  

#. Máximo de 20 executors que pueden lanzarse

spark.dynamicAllocation.maxExecutors 20   

 #. Si un executor temrina una tarea y permanece inactivo 2 minutos, se termina
 
spark.dynamicAllocation.executorIdleTimeout 2min    

Mencionemos que los números son arbitrarios y que los valores apropiados iran acorde a los workloads.



### Configuring Spark executors’ memory and the shuffle service

La asignación dinámica no es suficiente, hay que tener en cuenta como se ditribuye y utiliza la memoria del executor en Spark para que estos no se queden sin ella y se vean afectados por la JVM. 

La cantidad e memoria disponible a cada executor se controla mediante **spark.executor.memory.** Esta se divide en 3 secciones acorde a la siguiente figura:

- Execution Memory (60% default) / Memoria de ejecución. Es usada para los shuffles, joins, sorts y aggregations. Diferentes querys requieren difetentes fracciones de esta parte, pudiendo ser ajustada facilmente (spark.memory.fraction is 0.6 by default)
- Storage Memory (40% default) / Memoria de almacenamiento. Es usada para cachear datos estructurados y particiones derivadas de DFs.
- Reserved Memory (300MB default) / Memoria reservada


<center><img src="./images/Execution_memory_layout.PNG"></center>

Puede ajustar que fracción de spark.executor.memory quieres que utilice cualquiera de las dos secciones como línea de base. Si la Storage Memory no está siendo utilizada, Spark peude adquirirla para usarla como Execution Memory y viceversa. 

Saber que configuraciones por defecto son óptimas y ajustarlas puede ayudar a que no se provoquen cuellos de botella al hacer operaciones map y shuffle (que provocan gran actividad de I/O). Podemos ver en la siguiente tabla algunas recomendaciones  para coger ideas:

<center><img src="./images/map_and_shuffle_confs_ex.png"></center>

Ejemplo facebook https://towardsdatascience.com/how-does-facebook-tune-apache-spark-for-large-scale-workloads-3238ddda0830


### Maximizing Spark parallelism

La eficiencia de Spark tambien puede ver en la paralelización de multiples tareas a gran escala (Paralelizar es leer y procesar los datos lo más paralelamente posible). Miraremos como Spark lee los datos en memoria y qué significan las particiones para Spark.

En el ámbito del Data Management, una **partición** es una manera de organizar los datos en subconjuntos configurables y legibles por bloques en disco. Estos bloques pueden ser leidos/procesados independientemente y en paralelo. Esta independencia es importante por que permite el paralelismo masivo del procesamiento de los datos.

Spark es eficiente procesando tareas en paralelo. En large workloads scale, un job tendrá muchas etapas y dentro de estas muchas tareas. Spark programará (en el mejor de los casos), un hilo por tarea y cada tarea será procesasa en una particion distinta. 

Para optimizar los recursos y mazimizar el paralelismo es conveniendte tantas particiones como cores en cada Executor, estanto todos los cores ocupados. Puedes pensar en las particiones como unidades atómicas de paralelismo, un hilo corriendo por un core que puede trabajar en una única partición.

<center><img src="./images/cores_partition_paralelism.PNG"></center>

Las tareas de Spark procesan los datos como particiones, leidas desde el disco a la memoria. Estos datos en disco están divididos en bloques de archivos, dependiendo del store. Por defecto, los bloques de archivos en los almacenes tienen de 65mb a 128mm (ej en hdfs y amazon s3 por defecto es de 128mb). Una colección contigua de estos bloques forma una partición.

La propiedad que indica el tamaño de una partición es **spark.sql.files.maxPartitionBytes** (128 by default). Puedes reducir el tamaño pero se puede dar el "problema de los archivos pequeños", que indica que la relación de gran cantidad I/O en el disco baja el rendimiento gracias a las operaciones en el sistema de archivos (abrir/cerrar carpetas,archivos,etc) por lo que puede ser más lento.

Las particiones son creadas cuando usas ciertos metodos sobre un DF, por ejemplo, cuando lees o guardas un DF de gran tamaño desde el disco, puedes **ordenar a Spark que cree cierto número de particiones, con los metodos repartition y coalesce vistos en los ejercicios de word anteriores)**. Ejemplo:

In [18]:
# Cargamos DataFrame de M&M 
# Incicamos con repartition() el número de particiones que queremos que cree spark
mnm_df = ( spark.read.format("csv") 
                     .option("header", "true")
                     .option("inferSchema", "true")
                     .load("./Datasets/mnm_dataset.csv").repartition(16) )

# Consultamos el número de particiones de un DF con rdd.getNumePartitions
mnm_df.rdd.getNumPartitions()

16

Finalmente, las operaciones suffle en las particiones son creadas durante la fase shuffle. Por defecto, las shuffle partitions  son de 200 en la propiedad **spark.sql.shuffle.partitions**, pudiendo ser **ajustada en función del tamaño del DF o DS, cuanto más se reduzca, menos particiones son enviadas en read a los Executor para que realicen las tareas.**

**Por defecto el spark.sql.shuffle.partitions es muy alto para cargas de trabajo bajas, por lo que se conviene reducir este valor al número de cores de los Execurtors por lo menos.**

En las operaciones como group by o join (wide transformations), se consumen recursos de I/O de red y disco. Durante estas operaciones, el shuffle arrojará los resultados a los discos locales de los Executors en la ubicación especificada en la propiedad **spark.local.directory.** Tener discos SSD para este tipos de transformaciones mejora el rendimiento.

No existe una formula magica para el numero de shuffle partitions que establezca la fase de shuffle. Este dependerá de del caso de uso, DS, numero de cores, numero de executors disponibles en memoria, es una operación de prueba y error.

Referencias utiles:
https://www.youtube.com/watch?v=5dga0UT4RI8
https://www.youtube.com/watch?v=6BD-Vv-ViBw&t=645s
https://www.youtube.com/watch?v=6BD-Vv-ViBw&t=645s

## Caching and Persistence of Data

En large workloads, para mejorar el rendimiento, tendremos que considerar cachear o persistir nuestros DF o tablas en memoria con frecuencia. **¿Cúal es la diferencia entre Cachear y Persistir los datos?** En Spark, **son sinónimos**. Los 2 métodos: cache() y persist() ofrecen estas capacidades. 

persist() da más control sobre como y cuando los datos son almacenados, serializados y desserializados, pero las dos contribuyen a mejorar el rendimiento a la hora de acceder con frecuencia a DF o tablas.

### DataFrame.cache()

cache() almacena tantas particiones leídas **en memoria** entre los ejecutores de Spark como la memoria permita. Mientras que un DF puede ser cacheado de manera fraccionada, las particiones no pueden cacheadas así, por ejemplo, si mi DF tiene 8 particiones pero solo 4.5 pueden caber en memoria, solo 4 se almacenaran en cache. Por lo que, cuando tu quieras vovler a acceder a los datos, no todas las particiones estarán cacheadas y producirá mas lentidud en el Spark Job. 


In [44]:
# Cremos el dF
df = spark.range(1 * 10000000).toDF("id INT")
# Cacheamos
df.cache().explain()

# En este caso tarda mucho menos en hacer un count, por lo que el rendimiento mejora mucho
df.count()

== Physical Plan ==
*(1) ColumnarToRow
+- InMemoryTableScan [id INT#328L]
      +- InMemoryRelation [id INT#328L], StorageLevel(disk, memory, deserialized, 1 replicas)
            +- *(1) Project [id#190L AS id INT#192L]
               +- *(1) Range (0, 10000000, step=1, splits=12)




10000000

### DataFrame.persist()

persist() proporciona el control sobro como los datos son cacheados en el StorageLevel. En la siguiente tabal se pueden ver los diferentes niveles para este método. Por defecto, los datos en disco son serializados como Java o Kyro.


<center><img src="./images/storage_levels_persist().png"></center>

Mencionamos que cache() es lo mismo que persist(MEMORY_ONLY). Por lo que persist() nos ofrece mas niveles de cacheo.

Si queremos dejar de cachear un DF o tabla utilizamos el método unpersist()

Por ultimo, mencionamos que no solo se pueden cacechar los DF, tambien las vistas temporales usando SQLpuro.


In [45]:
df.createOrReplaceTempView("dfTable")
spark.sql("CACHE TABLE dfTable")
spark.sql("SELECT count(*) FROM dfTable").show()

+--------+
|count(1)|
+--------+
|10000000|
+--------+



### ¿Cuando cachear o persistir?

En los casos mas comunes vamos a querer acceder a un gran dataset repetidas veces para querys o transformaciones:
- DF usados de manera iterativa en Machine Learning
- DF que se acceden con frecuencia para transformaciones en operaciones ETL o construcción de data pipelines (automatizar el movimiento y transformación de los datos).

### ¿Cuando NO cachear o persistir?

Algunos escenarios que no será recomendable el uso de cacheo de DF:
- DFs que son demasiados grandes para ser almacenados en memoria
- Una trasnformación en un DF que no requiere un uso frecuente, independientemente de su tamaño.

Como regla general se debe utilizar la memoria caché con criterio, ya que puede incurrir en costes de recursos en la serialización y deserialización, dependiendo del StorageLevel utilizado.

## A Family of Spark Joins

Los Joins tambien producen movimientos costosos de datos, exigiendo recursos informaticos y de red del cluster. Veremos este movimiento a la hora de organizar los datos. De manera similar a una RBDMS, en un join se unen dos DF, tablas o DS. Es una operacion que produce grandes movimientos de datos sobre los Executors.

En le corazón de estas transformaciones se forma el shuffle. spark calcula que datos produce, escribe en disco, mueve hacia los nodos como parte de operaciones de agregado como groupby, agg, sort etc. Existen en Spark 5 tipos de Joins mediante el cual se intercambian, mueven, ordenan y agrupan los datos en los Executors, aunque nos centraremos en 2:
1. Broadcast hash join (BHJ)
2. Shuffle sort merge join (SMJ)





### 1 Broadcast hash join (BHJ)

Se emplea cuando 2 datasets, uno pequeño (que cabe en la memoria del driver y del executor) y otro lo suficientemente grande como para no tener que moverlo, necesitan ser unidos sobre ciertas condiciones o columnas. 

Se utiliza una variable de difusión y el DS mas pequeño es difundido por el driver a todos los executors. Posteriormente se une con el DS mas grande en cada executor. Se puede ver todo el proceso en la siguiente figura:

<center><img src="./images/bhj_join.png"></center>

Por defecto, spark utiliza un BHJ si el DS pequeño es menor de 10mb. Esta configuración se puede cambiar en la propiedad **spark.sql.autoBroadcastJoinThreshold**, pudiendo aumentar o disminuir dependiendo de la memoria que tengas para cada executor en el driver. Un valor de -1 en esta propiedas hará que spark haga un SMJ.

Un caso de uso común es cuando tienes varias keys entre 2 DF, uno mantiene menos información que el otro, y necesitas una vista combinada por ambas. Por ejemplo, consideremos un caso donde tienes un gran DS de jugadores de futbol (playersDF) y uno pequeño de clubes de futbol (clubsDF) y quieres unirlos por su clave correspondiente:

In [ ]:
from org.apache.spark.sql.functions import  broadcast
# NO EJECUTAR
joinedDF = playersDF.join(broadcast(clubsDF), "key1 === key2")

BHJ es el join más sencillo y facil que spark ofrece, ya que no implica ningun shuffle del DS; todos los datos están disponibles localmente para el Executor despues de su broadcast (emisión). Hay que estar seguro de que tienes suficiente memoria tanto para  el driver como en los executors para mantener el pequeño en memoria.

#### ¿Cuando usar un Broadcast hash join (BHJ)?
- Cuando cada key dentro de los DS pequeños y grandes estén en la misma particion
- Cuando un DS es mucho mas pequeño que el otro (dentro de 10mb o más si se tiene memoria)
- Cuando se desea combinar 2 DS basados en keys coincidentes sin clasificar
- Cuando no nos preocupa en exceso el ancho de banda o los OOM errors por que el DS pequeño será se puede transmitir por todos los ejecutores

### 2 Shuffle sort merge join (SMJ)

El algoritmo short-mege es una manera eficiente de unir 2 grandes DS con una clave comun que se puede ordenar, unificar y asignar a ser almacenado en la misma partición (esto signidica 2 DS con una misma clave que están en la misma partición). 

Spark entiende que todas las filas de cada conjunto de datos con la misma clave estan en la misma particion y el mismo executor, lo que indica que los datos tienen que ser colocados o intercambiados entre los executors. 

Como su nombre indica, tiene dos fases:
1. Fase de ordenado en cada DS por su clave de union deseada
2. Fase de unión que itera sobre cada clave en la columna de cada DS y une las columnas por las 2 claves.

Por defecto, la propiedad que indica el SorMeregeJoin (**spark.sql.join.preferSortMergeJoin.**) está activada. La idea principal es unir grnades DS por la columna key que se requiera.

Ejemplo pg 190--193



Podemos eliminar el paso de intercambio de este esquema si creamos buckets particionados para las claves ordenadas comunes o las columnas en las que queremos realizar equijoins frecuentes. Es decir, podemos crear un número explícito de buckets para almacenar columnas ordenadas específicas (una clave por bucket). 

La preclasificación y la reorganización de los datos de este modo aumentan el rendimiento, ya que nos permite omitir la costosa operación de intercambio e ir directamente a WholeStageCodegen.

En el siguiente ejemplo del cuaderno de este capítulo ordenamos y agrupamos por las columnas users_id y uid en las que y guardamos los buckets como tablas gestionadas por Spark en formato Parquet: (ejemplo 194-196)

#### ¿Cuando usar un shuffle sort merge join (SMJ)?
- Cuando cada clave en los 3 DS puede ser ordenada y hasheada por la misma partición
- Cuando queremos llevar a cabo equi-joins para combinar 2 DS basados en unir keys ordenadas
- Cuando queremos prevenir las operaciones de Intercambio y ordenado para guardar grandes shuffles sobre la red

Una vez visto esto nos centrarmos en la interfaz gráfica Spark UI para poder visualizar todo este tipo de operaciones que hemos ido viendo.

## Inspecting the Spark UI

Spark proporciona una elaborada interfaz web que nos permite inspeccionar varios componentes de nuestras aplicaciones. Ofrece detalles sobre el uso de memoria, trabajos, etapas y tareas, así como líneas de tiempo de eventos, registros y diversas métricas y estadísticas que pueden darnos una idea de lo que ocurre en nuestras aplicaciones Spark, tanto a nivel del Spark driver como en los executors individuales. Se puede conectar a la Spark UI en el puerto 4040 por defecto en local o a través del Spark Driver en otros modos.

## Spark UI Tabs

La spark UI (http://localhost:4040/ o 41,42 etc si tienes más spark sessions activas) está formada por 6 secciones que iremos viendo

### Jobs and Stages
Spark descompone una aplicación en jobs, stages y tareas. Las pestañas de Trabajos y Stages permiten navegar a través de ellas y profundizar a un nivel granular para examinar los detalles de las tareas individuales. Puede ver su estado de finalización
y revisar las métricas relacionadas con la I/O, el consumo de memoria, la duración de la ejecución etc.

La siguiente figura muestra la **pestaña Jobs** con la línea de tiempo de eventos expandida, mostrando cuando los executors fueron añadidos o eliminados del clúster. También proporciona una lista tabular de todos los trabajos completados en el cluster. 

La columna **Duración** indica el **tiempo que tardó cada trabajo** (identificado por el ID del trabajo en la primera columna) en terminar. **Si este tiempo es alto**, es una **indicación de que podría querer investigar las etapas de ese trabajo para ver qué tareas pueden estar causando retrasos.** Desde esta página de resumen también puede acceder a una página de detalles de cada trabajo, que incluye una visualización del DAG y una lista de las etapas completadas.

<center><img src="./images/jobsUI.png"></center>

La **pestaña Stages** proporciona un **resumen del estado actual de todas las stages de todos los trabajos en la aplicación.**
También puede acceder a una página de detalles para cada etapa, que proporciona un DAG y métricas sobre sus tareas.

Además de otras **estadísticas** opcionales, puede ver la duración media de cada tarea, el tiempo empleado en la recolección de basura (GC), y número de bytes/registros leídos. **Si los datos del shuffle están siendo leídos desde executors remotos, un alto tiempo de bloqueo en la lectura del shuffle puede indicar problemas de I/O.**

Un **tiempo de GC alto indica demasiados objetos en la pila (hace falta memoria en los executors)**. Si el **tiempo máximo de una tarea es mucho más grande que la mediana**, entonces es probable que tenga un **sesgo de datos causada por una distribución desigual de los datos en sus particiones**. Busque estos signos reveladores.

<center><img src="./images/stages_UI.png"></center>



### Executors
Esta pestaña proporciona **información de los executors creados por la aplicación**. Puedes profundizar en numerosos detalles sobre el uso de recursos (disco, memoria, cores), tiempo gastado en DC, cantidad de datos escritos/leidos durante el shuffle, etc.

Aparte del resumen estadístico, puedes **ver como se utiliza la memoria por cada executor individualmente y con qué proposito**. Tambien ayuda a **examinar el uso de recursos cuando se ha utilizado el método cache() o persist en un DF o tabla**.

<center><img src="./images/executors_UI.png"></center>


### Storage

Esta pestaña da **información sobre tablas o DF cacheados por la aplicación como resultado de los metodos cache() y persist().**

Pulsando en el RDD Name correspondiente a la tabla o DF cacheada/persistida, podemos ver como el DF o tabla ha sido cacheada en memoria y disco sobre el executor y sus partciciones correspondientes (el numero de particiones corresponde al numero de buckets fijado).

### SQL

Podemos ver los **efectos de las querys SQL de spark y como se ejcutan, siendo rastreables y visibles en esta pestaña. Se puede ver cuando se ejecutaron las querys, su job y su duración.**

Pulsando en una query (columna description) podemos ver mas detalles del plan de ejecución con detalle. Es de utilidad cundo qurremos inspeccionar detalles de un plan fisico.

### Enviroment

Esta pestaña muestra el entorno en el que se ejcuta la aplicación de spark, siendo util para la resolución de problemas. Qué varables de entorno, que jars se han incluido, propiedades establecidas y sus respectivos valores, entorno de ejecucion ,etc. Todo esto se puede consultar en detalle, sinedo muy util en el caso de que notes algún comportamiento extraño en Spark.

### Debugging Spark applications

Para debuggear o depurar se lleva a cabo en tu IDE favorito, el libro recomienda IntelliJ IDEA en un local host **(mirar si se puede mover en python)*